In [11]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import copy
import numpy as np
import random
from SimpleCNN import SimpleCNNArchitecture

# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
seed = 42
set_seed(seed)

# Directories
image_dir = r'C:\Users\guang\OneDrive\Desktop\IMDA\torch_training_data\letters\input'
label_dir = r'C:\Users\guang\OneDrive\Desktop\IMDA\torch_training_data\letters\output'

# Custom Dataset
class CharacterDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = glob.glob(os.path.join(image_dir, '*.jpg'))
        self.label_files = glob.glob(os.path.join(label_dir, '*.txt'))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label_path = os.path.join(self.label_dir, os.path.basename(img_path.replace("in", "out")).replace('.jpg', '.txt'))
        
        image = Image.open(img_path).convert('L')  # Convert image to grayscale
        with open(label_path, 'r') as f:
            label = f.read().strip()

        if self.transform:
            image = self.transform(image)

        label = ord(label) - ord('A') if 'A' <= label <= 'Z' else ord(label) - ord('0') + 26
        return image, label


# Function to compute mean and std
def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)
    mean = 0.0
    std = 0.0
    total_samples = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_samples += batch_samples

    mean /= total_samples
    std /= total_samples

    return mean, std

# Create a dataset without normalization to compute mean and std
dataset = CharacterDataset(image_dir, label_dir, transform=transforms.ToTensor())
mean, std = compute_mean_std(dataset)
print(f"Mean: {mean}, Std: {std}")


# Data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Create dataset
dataset = CharacterDataset(image_dir, label_dir, transform=transform)

# Perform train/validation/test split
train_size = int(0.6 * len(dataset))
validation_size = len(dataset) - train_size
train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

# Data loaders
batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# validation_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)

# Model, loss function, optimizer
model = SimpleCNNArchitecture()
criterion = nn.CrossEntropyLoss()
learn_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learn_rate)

# Lambda function for the scheduler
def lr_lambda(epoch):
    return max(1 - epoch / 300, 0.01)

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)


# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_acc = 0.0
    best_model = copy.deepcopy(model)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # Step the scheduler
        if scheduler != None:
            scheduler.step()
        epoch_loss = running_loss
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_loss 
        val_accuracy = correct / total
        if val_accuracy > best_val_acc:
            best_model = copy.deepcopy(model)
            best_val_acc = val_accuracy
            print("!!!NEW BEST VAL ACC", best_val_acc)
            print()
        print(f'Current Best Val Acc: {best_val_acc:.4f} Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
    best_model.eval()
    return best_model

# Evaluate the model on the test set
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')
    # Assuming 'model' is your trained model
    

Mean: tensor([0.6465]), Std: tensor([0.4210])


In [13]:
# Train the model
best_model = train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=300)

# Evaluate
evaluate_model(best_model, validation_loader)

# Save the model
best_model.eval()
torch.save(best_model.state_dict(), 'imda_technical_test_pytorch_model.pth')


Epoch 1/300, Loss: 260.8895
!!!NEW BEST VAL ACC 0.020833333333333332

Current Best Val Acc: 0.0208 Validation Loss: 171.0707, Validation Accuracy: 0.0208
Epoch 2/300, Loss: 242.8272
!!!NEW BEST VAL ACC 0.1875

Current Best Val Acc: 0.1875 Validation Loss: 168.9267, Validation Accuracy: 0.1875
Epoch 3/300, Loss: 215.5373
!!!NEW BEST VAL ACC 0.22916666666666666

Current Best Val Acc: 0.2292 Validation Loss: 146.5805, Validation Accuracy: 0.2292
Epoch 4/300, Loss: 157.2504
!!!NEW BEST VAL ACC 0.3125

Current Best Val Acc: 0.3125 Validation Loss: 126.5211, Validation Accuracy: 0.3125
Epoch 5/300, Loss: 104.6676
!!!NEW BEST VAL ACC 0.4583333333333333

Current Best Val Acc: 0.4583 Validation Loss: 101.9311, Validation Accuracy: 0.4583
Epoch 6/300, Loss: 72.1845
!!!NEW BEST VAL ACC 0.6458333333333334

Current Best Val Acc: 0.6458 Validation Loss: 83.9231, Validation Accuracy: 0.6458
Epoch 7/300, Loss: 42.4728
!!!NEW BEST VAL ACC 0.875

Current Best Val Acc: 0.8750 Validation Loss: 63.2507, Va

In [14]:
# Sample Inference of torch trained torch model
from TorchInferenceEngine import load_torch_ai_model
from PIL import Image

torch_ai_model = load_torch_ai_model(model_path = 'imda_technical_test_pytorch_model.pth')
sample_catpcha_image_path = r'C:\Users\guang\OneDrive\Desktop\IMDA\sampleCaptchas\input\input17.jpg'
sample_catpcha_image_path_2 = r'C:\Users\guang\OneDrive\Desktop\IMDA\torch_training_data\reserved_unseen\input100.jpg'

ir = torch_ai_model.get_image_information(sample_catpcha_image_path)
ir2 = torch_ai_model.get_image_information(sample_catpcha_image_path_2)
print(ir, ir2)

sample_image = Image.open(sample_catpcha_image_path)
display(sample_image)

sample_image_2 = Image.open(sample_catpcha_image_path_2)
display(sample_image_2)

{'InferredCharacters': '20BHQ'} {'InferredCharacters': 'YMB1Q'}


C:\Users\guang\OneDrive\Desktop\IMDA\TorchInferenceEngine.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch_model.load_state_dict(torch.load(model_path))


In [17]:
# The Torch Model without O0I1 domain specific discrimination is already capable enough. But,
# lets train a O0I1 Discriminator by using the best_model as the pre-trained starting point
# Directories
O01I_image_dir = r'C:\Users\guang\OneDrive\Desktop\IMDA\torch_training_data\letters_only_O01I\input'
O01I_label_dir = r'C:\Users\guang\OneDrive\Desktop\IMDA\torch_training_data\letters_only_O01I\output'

# Create dataset
O01I_dataset = CharacterDataset(O01I_image_dir, O01I_label_dir, transform=transform)

# Data loaders
batch_size = 1
O01I_train_loader = DataLoader(O01I_dataset, batch_size=batch_size, shuffle=True)
O01I_validation_loader = O01I_train_loader
O01I_criterion = nn.CrossEntropyLoss()
O01I_learn_rate = 0.0001
O01I_optimizer = optim.Adam(model.parameters(), lr=O01I_learn_rate)
scheduler = None

# Train
O01I_best_model = train_model(best_model, O01I_train_loader, O01I_validation_loader, O01I_criterion, O01I_optimizer, num_epochs=10)

# Save
O01I_best_model.eval()
torch.save(O01I_best_model.state_dict(), 'imda_technical_test_pytorch_O01IDiscriminator_model.pth')

Epoch 1/10, Loss: 0.8182
!!!NEW BEST VAL ACC 1.0

Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 2/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 3/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 4/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 5/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 6/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 7/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 8/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 9/10, Loss: 0.8182
Current Best Val Acc: 1.0000 Validation Loss: 0.8182, Validation Accuracy: 1.0000
Epoch 10/10,

In [19]:
# Sample Inference of O01I discriminator torch trained torch model
from TorchInferenceEngine import load_torch_ai_model
from PIL import Image

torch_ai_model = load_torch_ai_model(model_path = 'imda_technical_test_pytorch_O01IDiscriminator_model.pth')
sample_catpcha_image_path = r'C:\Users\guang\OneDrive\Desktop\IMDA\sampleCaptchas\input\input17.jpg'
sample_catpcha_image_path_2 = r'C:\Users\guang\OneDrive\Desktop\IMDA\torch_training_data\reserved_unseen\input100.jpg'

ir = torch_ai_model.get_image_information(sample_catpcha_image_path)
ir2 = torch_ai_model.get_image_information(sample_catpcha_image_path_2)
print(ir, ir2)

sample_image = Image.open(sample_catpcha_image_path)
display(sample_image)

sample_image_2 = Image.open(sample_catpcha_image_path_2)
display(sample_image_2)

{'InferredCharacters': '20BHQ'} {'InferredCharacters': 'YMB1Q'}


In [ ]:
# !pip3 install torch torchvision torchaudio